In [ ]:
!pip install tensorflow_text==2.7.3 
!pip install tensorflow==2.7.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 8.0 MB/s 
     - 671.4 MB 273 kB/s
     |████████████████████████████████| 463 kB 28.3 MB/s 
     |████████████████████████████████| 1.3 MB 46.2 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5.3
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.8.0+zzzcolab20220506162203
    Uninstalling tensorflow-2.8.0+zzzcolab20220506162203:
      Successfully uninstalled tensor

In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-05-28 20:44:00--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-05-28 20:44:00 (101 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [ ]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [ ]:
from tensorflow.keras import layers

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split


In [ ]:
from __future__ import division, print_function
from keras.callbacks import ModelCheckpoint
from keras.callbacks import History
from keras.layers import Dense, Dropout, BatchNormalization
from keras.models import Sequential
from keras.models import Model

In [ ]:
import tensorflow.compat.v1 as tf 
import tensorflow_hub as hub
import tensorflow_text
import pandas as pd
import numpy as np


In [ ]:

data = pd.read_csv('/content/drive/MyDrive/НИР/Join_dataset.csv')



data_train, data_valid, train_labels, valid_labels = train_test_split(data["text"],
                                         data["news"],                              
                                         test_size=0.3, 
                                         random_state=42)

data_valid = data_valid.to_numpy()
valid_labels = valid_labels.to_numpy()

data_train, data_test, train_labels, test_labels = train_test_split(data_train.to_numpy(), 
                                         train_labels.to_numpy(),                           
                                         test_size=0.143, 
                                         random_state=42)

test_vk = pd.read_csv('/content/drive/MyDrive/НИР/Join_dataset_VK_NEWest_2.csv')
x_test_vk = test_vk['text']
y_test_vk = test_vk['news']
#y_test_vk = [str(t) for t in y_test_vk]

In [ ]:
model_url = 'https://tfhub.dev/google/universal-sentence-encoder-multilingual/3' 
embed = hub.load(model_url)

def embed_text(input):
  return embed(input)

In [ ]:
sentence_encoder_layer = hub.KerasLayer(model_url,
                                        input_shape=[], # shape of inputs coming to our model 
                                        dtype=tf.string, # data type of inputs coming to the USE layer
                                        trainable=False, # keep the pretrained weights (we'll create a feature extractor)
                                        name="USE")

In [ ]:
classifier_USE = tf.keras.Sequential([
  sentence_encoder_layer, # take in sentences and then encode them into an embedding
  layers.Dense(64, activation="relu"),
  layers.Dense(1, activation="sigmoid")
], name="classifier_USE")

# Compile model
classifier_USE.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

classifier_USE.evaluate(x = data_valid, y = valid_labels)
classifier_USE.summary()

94/94 [==============================] - 20s 173ms/step - loss: 0.6964 - accuracy: 0.4457
Model: "classifier_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               68927232  
                                                                 
 dense (Dense)               (None, 64)                32832     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 68,960,129
Trainable params: 32,897
Non-trainable params: 68,927,232
_________________________________________________________________


In [ ]:
SAVE_DIR = "/content/drive/MyDrive/НИР/models"


In [ ]:
classifier_probs_pre = classifier_USE.predict(data_test)
classifier_preds_pre = tf.squeeze(tf.round(classifier_probs_pre))


In [ ]:
# Train a classifier on top of pretrained embeddings
 
history = History()
all_valid_loss = []
all_train_loss = []
all_valid_acc = []
all_train_acc = []

classifier_USE.fit(data_train,
                              train_labels,
                              epochs=6,
                              validation_data=(data_valid, valid_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, 
                                                                     "tf_hub_sentence_encoder"), history])



Saving TensorBoard log files to: /content/drive/MyDrive/НИР/models/tf_hub_sentence_encoder/20220528-204446
Epoch 1/6
188/188 [==============================] - 37s 168ms/step - loss: 0.2842 - accuracy: 0.9475 - val_loss: 0.1130 - val_accuracy: 0.9693
Epoch 2/6
188/188 [==============================] - 31s 167ms/step - loss: 0.0856 - accuracy: 0.9765 - val_loss: 0.0840 - val_accuracy: 0.9747
Epoch 3/6
188/188 [==============================] - 33s 178ms/step - loss: 0.0642 - accuracy: 0.9802 - val_loss: 0.0761 - val_accuracy: 0.9753
Epoch 4/6
188/188 [==============================] - 32s 169ms/step - loss: 0.0533 - accuracy: 0.9842 - val_loss: 0.0737 - val_accuracy: 0.9767
Epoch 5/6
188/188 [==============================] - 32s 170ms/step - loss: 0.0464 - accuracy: 0.9867 - val_loss: 0.0728 - val_accuracy: 0.9760
Epoch 6/6
188/188 [==============================] - 32s 170ms/step - loss: 0.0417 - accuracy: 0.9873 - val_loss: 0.0729 - val_accuracy: 0.9773


In [ ]:
all_valid_loss = history.history['val_loss']
all_train_loss = history.history['loss']
all_valid_acc = history.history['val_accuracy']
all_train_acc = history.history['accuracy']

In [ ]:
classifier_probs = classifier_USE.predict(data_test)

In [ ]:
classifier_preds = tf.squeeze(tf.round(classifier_probs))

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):

  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred)
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [ ]:
classifier_VK = classifier_USE.predict(x_test_vk)
classifier_vk = tf.squeeze(tf.round(classifier_VK))

In [ ]:
classifier_results_pre = calculate_results(test_labels, classifier_preds_pre)
classifier_results = calculate_results(test_labels, classifier_preds)
print(classifier_results_pre)
print(classifier_results)
classifier_results_VK = calculate_results(y_test_vk, classifier_vk)
print(classifier_results_VK)

{'accuracy': 41.95804195804196, 'precision': array([0.41965471, 0.41935484]), 'recall': array([0.68695652, 0.1922366 ]), 'f1': array([0.52102226, 0.26362484])}
{'accuracy': 98.30169830169831, 'precision': array([0.98896247, 0.97810219]), 'recall': array([0.97391304, 0.99075786]), 'f1': array([0.98138007, 0.98438935])}
{'accuracy': 91.85, 'precision': array([0.98719441, 0.86678352]), 'recall': array([0.848, 0.989]), 'f1': array([0.91231845, 0.92386735])}


In [ ]:
!pip install plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import plotly.graph_objects as go

#print('Originaly loss of the model:', valid_loss_ep)

fig = go.Figure(
    data=[go.Line(y = all_train_loss, x=[1,2,3,4,5,6])]
)
fig.update_layout(title='Train Loss Multilingual USE',
                   xaxis_title='Epoch',
                   yaxis_title='Loss')

fig_2 = go.Figure(
    data=[go.Line(y = all_valid_loss, x=[1,2,3,4,5,6])]
)
fig_2.update_layout(title='Valid Loss Multilingual USE',
                   xaxis_title='Epoch',
                   yaxis_title='Loss')

fig.show(renderer="colab")
fig_2.show(renderer="colab")

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning: plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.

  DeprecationWarning,
/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [ ]:
fig = go.Figure(
    data=[go.Line(y = all_train_acc, x=[1,2,3,4,5,6])]
)

fig.update_layout(title='Train Accuracy Multilingual USE',
                   xaxis_title='Epoch',
                   yaxis_title='Accuracy')

fig_2 = go.Figure(
    data=[go.Line(y = all_valid_acc, x=[1,2,3,4,5,6])]
)
fig_2.update_layout(title='Valid Accuracy Multilingual USE',
                   xaxis_title='Epoch',
                   yaxis_title='Accuracy')

fig.show(renderer="colab")
fig_2.show(renderer="colab")


/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


